In [1]:
import import_ipynb
from loadData import raw_df, ad_info_columns, user_info_columns, context_info_columns, shop_info_columns, result_columns,ad_info_df, user_info_df, context_info_df, shop_info_df, result_df
# from EDA import draw_df_feature_frequency
from Util import getAllTypesofCategory, getAllTypesOfProperty

importing Jupyter notebook from Util.ipynb


In [65]:
from sklearn.preprocessing import Imputer, MinMaxScaler, OneHotEncoder
import numpy as np
import itertools
import pandas as pd
import collections

## impute missing value with most_frequent strategy

In [3]:
contain_missing_value_columns = ["item_brand_id","item_city_id","item_sales_level","user_gender_id","user_age_level",
                                "user_occupation_id","user_star_level","shop_review_positive_rate",
                                "shop_score_service","shop_score_delivery","shop_score_description"]
other_missing_value_columns = ["predict_category_property"]

In [4]:
data_imputer = Imputer(missing_values=-1 , strategy='most_frequent', axis=0)
raw_df[contain_missing_value_columns] = data_imputer.fit_transform(raw_df[contain_missing_value_columns])

## processing the object column

In [5]:
object_columns = ["item_category_list", "item_property_list", "predict_category_property"]
raw_df[object_columns].head()

,item_category_list,item_property_list,predict_category_property
0,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,5799347067982556520:-1;509660095530134768:-1;5...
1,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,5799347067982556520:9172976955054793469;790838...
2,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,5799347067982556520:5131280576272319091;725801...
3,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,509660095530134768:-1;5799347067982556520:-1;7...
4,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,5799347067982556520:9172976955054793469;790838...


In [34]:
# split category_list to multi features
def extract_item_category_list_func(item, num):
    items = item.split(';')
    if num >=2:
        if len(items) > 2:
            return items[num]
        else:
            return 0
    else:
        return items[num]
def generate_item_category_features():
    raw_df['item_one_level_category'] = raw_df['item_category_list'].apply(lambda item: extract_item_category_list_func(item, 0))
    raw_df['item_second_level_category'] = raw_df['item_category_list'].apply(lambda item: extract_item_category_list_func(item, 1))
    raw_df['item_third_level_category'] = raw_df['item_category_list'].apply(lambda item: extract_item_category_list_func(item, 2))
#     raw_df.drop('item_category_list', axis=1)
generate_item_category_features()

In [35]:
# transform property_list
extract_item_property_list_func = lambda item: list(item.split(';'))
raw_df['item_property_list_array'] = raw_df['item_property_list'].apply(lambda item: extract_item_property_list_func(item))

In [36]:
arrays = list(raw_df['item_property_list_array'])
count = collections.Counter(list(itertools.chain(*arrays)))
most_common_property_id_list = list(list(zip(*count.most_common(1499)))[0])
most_common_property_id_list.append('others')

## generate new raw_df called _raw_df after filling missing value and handle category_id and property _id, drop useless column

In [38]:
_raw_df = raw_df.drop(['item_category_list','item_property_list'], axis=1)

In [57]:
need_onehot_columns = ['item_brand_id','item_city_id','user_gender_id','item_one_level_category',
                       'item_second_level_category']
special_onehot_columns = ['item_property_list_array', 'item_third_level_category']
other_columns = ['context_timestamp','predict_category_property','is_trade']
need_scale_columns = ['item_price_level','item_sales_level','item_collected_level','item_pv_level',
                     'user_age_level','user_occupation_id','user_star_level','context_page_id',
                     'shop_review_num_level','shop_review_positive_rate','shop_star_level','shop_score_service',
                    'shop_score_description']

In [49]:
#transform to label type
for i in ['item_brand_id','item_city_id','user_gender_id']:
    _raw_df[i] = _raw_df[i].apply(lambda item : str(item))

## One hot encoding

In [51]:
need_onehot_df = pd.get_dummies(_raw_df[need_onehot_columns], prefix=need_onehot_columns)

## scaling data in need_scale_column

In [62]:
# min-max normalization
_tmpdf = _raw_df[need_scale_columns]
# normalized_df = (_tmpdf - _tmpdf.min())/(_tmpdf.max() - _tmpdf.min())

# mean normalization
normalized_df = (_tmpdf - _tmpdf.mean())/_tmpdf.std()

In [69]:
OneHotEncoder?